In [1]:
#导入数据
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time
#展示所有行
pd.set_option('display.max_columns', None)

ModuleNotFoundError: No module named 'seaborn'

In [ ]:
#读取training_data
data = pd.read_csv('D:/Fintec_Class/ML/ml_case_data/ml_case_training_data.csv')
data.head()

In [ ]:
#将数据中的零值变为空值
data = data.replace(0,np.nan)

In [ ]:
data.info()

In [ ]:
#删除缺失值达到20%的变量
val_num = len(data.columns)
val_miss = {}
val_all = data.isnull().sum()
for i in range(val_num):
    if val_all[i] != 0:
        val_miss[data.columns[i]]=val_all[i]
val_miss_names = list(val_miss.keys())
data_num = len(data)
for i in range(len(val_miss)):
    if val_miss[val_miss_names[i]]>0.2*data_num:
        print('删除'+val_miss_names[i])
        data = data.drop(val_miss_names[i],axis=1) 

In [ ]:
data.info()

In [ ]:
data['date_end'].isnull()[2]

In [ ]:
#求合同开始和结束的间隔天数，单独的日期没有实际意义。
data['days']=0
def cal_days(data):
    for i in range(len(data)):
        if data['date_end'].isnull()[i]==False:
            start_time = data['date_activ'][i]
            end_time = data['date_end'][i]
            start = time.mktime(time.strptime(start_time,'%Y-%m-%d'))
            end = time.mktime(time.strptime(end_time,'%Y-%m-%d'))
            data['days'][i] = int((end - start)/(24*60*60))
        else:
            data['days'][i] = np.nan
cal_days(data)
#data.drop(['date_activ','date_end'],axis=1,inplace=True)
data.head()

In [ ]:
#求下一个续约日和结束的间隔天数，单独的日期没有实际意义。
data['renewal_days']=0
def cal_days(data):
    for i in range(len(data)):
        if (data['date_end'].isnull()[i]==False) and (data['date_renewal'].isnull()[i]==False):
            renewal_time = data['date_renewal'][i]
            end_time = data['date_end'][i]
            renewal = time.mktime(time.strptime(renewal_time,'%Y-%m-%d'))
            end = time.mktime(time.strptime(end_time,'%Y-%m-%d'))
            data['renewal_days'][i] = int((renewal-end)/(24*60*60))
        else:
            data['renewal_days'][i] = np.nan
cal_days(data)
#data.drop(['renewal_days','date_end'],axis=1,inplace=True)
data.head(60)

In [ ]:
#判断最后一次修改处于什么阶段
#当在开始日期之前为0
#当等于开始日期时为1
#当在合同执行日期之时为2
#当超出合同日期为3
data['modif_label']=0
def cal_modif_label(data):
    for i in range(len(data)):
        if (data['date_end'].isnull()[i]==False) and (data['date_modif_prod'].isnull()[i]==False):
            start_time = data['date_activ'][i]
            date_modif_prod = data['date_modif_prod'][i]
            end_time = data['date_end'][i]
            modif = time.mktime(time.strptime(date_modif_prod,'%Y-%m-%d'))
            start = time.mktime(time.strptime(start_time,'%Y-%m-%d'))
            end = time.mktime(time.strptime(end_time,'%Y-%m-%d'))
            if modif<start:
                data['modif_label'][i] = 0
            elif modif==start:
                data['modif_label'][i] = 1
            elif modif>start and modif<=end:
                data['modif_label'][i] = 2
            elif modif>end:
                data['modif_label'][i] = 3
        else:
            data['modif_label'][i] = np.nan
cal_modif_label(data)
#data.drop(['renewal_days','date_end'],axis=1,inplace=True)
data.head(60)

In [ ]:
origin_list = data['origin_up'].unique().tolist()
def change_list(data,origin_list):
    j = 0
    for i in origin_list:
        data.loc[data['origin_up']==i,'origin_up']=j
        j +=1
change_list(data,origin_list)
data.head()

In [ ]:
data.loc[data['has_gas']=='t','has_gas']=1
data.loc[data['has_gas']=='f','has_gas']=0

In [ ]:
#判断数据的数据类型，决定采用哪种插值方法
#data['pow_max'].value_counts()
#data['origin_up'].value_counts()
#data['date_end'].value_counts()
data['date_modif_prod'].value_counts()

In [ ]:
data.info()

In [ ]:
#根据数据类型的不同，采用不同的方式填充数据
data.loc[data['cons_12m'].isnull(),'cons_12m']=data['cons_12m'].median()
data.loc[data['forecast_cons_12m'].isnull(),'forecast_cons_12m']=data['forecast_cons_12m'].median()
data.loc[data['forecast_meter_rent_12m'].isnull(),'forecast_meter_rent_12m']=data['forecast_meter_rent_12m'].median()
data.loc[data['forecast_price_energy_p1'].isnull(),'forecast_price_energy_p1']=data['forecast_price_energy_p1'].median()
data.loc[data['forecast_price_pow_p1'].isnull(),'forecast_price_pow_p1']=data['forecast_price_pow_p1'].median()
data.loc[data['net_margin'].isnull(),'net_margin']=data['net_margin'].median()
data.loc[data['margin_gross_pow_ele'].isnull(),'margin_gross_pow_ele']=data['margin_gross_pow_ele'].median()
data.loc[data['margin_net_pow_ele'].isnull(),'margin_net_pow_ele']=data['margin_net_pow_ele'].median()
data.loc[data['pow_max'].isnull(),'pow_max']=data['pow_max'].median()
data.loc[data['days'].isnull(),'days']=data['days'].median()
data.loc[data['renewal_days'].isnull(),'renewal_days']=data['renewal_days'].median()
data.loc[data['modif_label'].isnull(),'modif_label']=data['modif_label'].mode()[0]
data.loc[data['origin_up'].isnull(),'origin_up']=data['origin_up'].mode()[0]

In [ ]:
data.info()

In [ ]:
data.drop(['date_activ','date_end','date_modif_prod','date_renewal'],axis=1,inplace=True)
data.info()

In [ ]:
#读取hist_data数据
hist_data = pd.read_csv('D:/Fintec_Class/ML/ml_case_data/ml_case_training_hist_data.csv')
hist_data.head()

In [ ]:
#将数据中的零值变为空值
hist_data = hist_data.replace(0,np.nan)

In [ ]:
#数据基础信息，有多少条
hist_data.info()

In [ ]:
#删除缺失值达到20%的变量
val_num = len(hist_data.columns)
val_miss = {}
val_all = hist_data.isnull().sum()
for i in range(val_num):
    if val_all[i] != 0:
        val_miss[hist_data.columns[i]]=val_all[i]
val_miss_names = list(val_miss.keys())
data_num = len(hist_data)
for i in range(len(val_miss)):
    if val_miss[val_miss_names[i]]>0.2*data_num:
        print('删除'+val_miss_names[i])
        hist_data = hist_data.drop(val_miss_names[i],axis=1) 
hist_data.info()

In [ ]:
#按照id分组，因为这个数据是每个id每12个月的数据，其他数据集每个id只有一条数据。所以求价格均值，展现一条数据，方便处理
hist_data = hist_data.groupby('id',as_index=False).mean()
hist_data.head()

In [ ]:
#求均值后舍弃其他数据
#hist_data.drop(['price_p1_var','price_p2_var','price_p3_var','price_p1_fix','price_p2_fix','price_p3_fix'],axis=1,inplace=True)
hist_data.info()

In [ ]:
#有一行有缺失值，用中位数填充
hist_data.loc[hist_data['price_p1_var'].isnull(),'price_p1_var']=hist_data['price_p1_var'].median()
hist_data.loc[hist_data['price_p1_fix'].isnull(),'price_p1_fix']=hist_data['price_p1_fix'].median()
hist_data.info()

In [ ]:
#读取output数据
output_data = pd.read_csv('D:/Fintec_Class/ML/ml_case_data/ml_case_training_output.csv')
output_data.head()

In [ ]:
output_data.info()

In [ ]:
#对data和hist_data进行合并操作，内连接合并
data = pd.merge(data,hist_data,on=['id'],how='inner')
data.head()
data.info()

In [ ]:
#对data和output_data进行合并操作，内连接合并
data = pd.merge(data,output_data,on=['id'],how='inner')
data.head()
data.info()

In [ ]:
data.head()

In [ ]:
#查看标签情况
f,ax=plt.subplots(1,2,figsize=(18,8))
data['churn'].value_counts().plot.pie(explode=[0,0.1],autopct='%1.1f%%',ax=ax[0],shadow=True)
ax[0].set_title('churn')
ax[0].set_ylabel('')
sns.countplot('churn',data=data,ax=ax[1])
ax[1].set_title('churn')
plt.show()

In [ ]:
#has_gas与标签关系
f,ax=plt.subplots(1,2,figsize=(18,8))
data[['has_gas','churn']].groupby(['has_gas']).mean().plot.bar(ax=ax[0])
ax[0].set_title('churn vs has_gas')
sns.countplot('has_gas',hue='churn',data=data,ax=ax[1])
ax[1].set_title('has_gas:1 vs 0')
plt.show()

In [ ]:
data.head()

In [ ]:
#data['price_p1_var_Range']=pd.qcut(data['price_p1_var'],5)
#data.groupby(['price_p1_var_Range'])['churn'].mean().to_frame().style.background_gradient(cmap='summer_r')

In [ ]:
def qcut_columns(data):
    data['Fare_Range']=pd.qcut(data['Fare'],5)
    data.groupby(['Fare_Range'])['Survived'].mean().to_frame().style.background_gradient(cmap='summer_r')

In [ ]:
data.drop(['cons_12m_Range','price_p1_var_Range'],axis=1,inplace=True)
data.head()

In [ ]:
#importing all the required ML packages
from sklearn.linear_model import LogisticRegression #logistic regression
from sklearn import svm #support vector Machine
from sklearn.ensemble import RandomForestClassifier #Random Forest
from sklearn.neighbors import KNeighborsClassifier #KNN
from sklearn.naive_bayes import GaussianNB #Naive bayes
from sklearn.tree import DecisionTreeClassifier #Decision Tree
from sklearn.model_selection import train_test_split #training and testing data split
from sklearn import metrics #accuracy measure
from sklearn.metrics import confusion_matrix #for confusion matrix

In [ ]:
train,test=train_test_split(data,test_size=0.3,random_state=0,stratify=data['churn'])
train_X=train[train.columns[1:]]
train_Y=train[train.columns[:1]]
test_X=test[test.columns[1:]]
test_Y=test[test.columns[:1]]
X=data[data.columns[1:]]
Y=data['churn']

In [ ]:
model=svm.SVC(kernel='rbf',C=1,gamma=0.1)
model.fit(train_X,train_Y)
prediction1=model.predict(test_X)
print('Accuracy for rbf SVM is ',metrics.accuracy_score(prediction1,test_Y))

In [ ]:
model=RandomForestClassifier(n_estimators=100)
model.fit(train_X,train_Y)
prediction7=model.predict(test_X)
print('The accuracy of the Random Forests is',metrics.accuracy_score(prediction7,test_Y))

In [ ]:
model=DecisionTreeClassifier()
model.fit(train_X,train_Y)
prediction4=model.predict(test_X)
print('The accuracy of the Decision Tree is',metrics.accuracy_score(prediction4,test_Y))